In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd '/content/drive/MyDrive/2022OME'
%pwd

/content/drive/MyDrive/2022OME


'/content/drive/MyDrive/2022OME'

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [4]:
import os
from PIL import Image
import shutil
from tqdm.auto import tqdm

In [5]:
#list = [left, upper, right, lower]
#formal edition
block_pos = {
    "block1":[964, 660, 1182, 1092],
    "block2":[1222, 667, 1713, 1014],
    "block3":[1853, 667, 2359, 1022],
    "block4":[2866, 704, 3115, 1101],
    "block5":[1025, 1332, 1090, 1397],
    "block6":[1017, 1403, 1083, 1470],
    "block7":[1015, 1476, 1092, 1539],
    "block8":[1025, 1545, 1084, 1606],
    "block9":[1013, 1605, 1087, 1675],
    "block10":[1097, 1332, 1184, 1390],
    "block11":[1084, 1394, 1189, 1462],
    "block12":[1098, 1469, 1185, 1539],
    "block13":[1104, 1536, 1188, 1599],
    "block14":[1087, 1603, 1180, 1675],
    "block15":[1209, 1301, 1331, 1667],
    "block16":[1331, 1317, 1451, 1656],
    "block17":[1554, 1368, 1638, 1448],
    "block18":[1561, 1451, 1633, 1519],
    "block19":[1561, 1522, 1631, 1582],
    "block20":[1551, 1583, 1632, 1657],
    "block21":[1652, 1376, 1725, 1446],
    "block22":[1639, 1449, 1731, 1520],
    "block23":[1643, 1514, 1721, 1590],
    "block24":[1647, 1591, 1724, 1653],
    "block25":[1746, 1373, 1812, 1441],
    "block26":[1742, 1443, 1817, 1518],
    "block27":[1745, 1515, 1811, 1587],
    "block28":[1740, 1589, 1809, 1656],
    "block29":[1836, 1336, 1872, 1636],
    "block30":[1873, 1333, 1910, 1631],
    "block31":[1971, 1159, 2190, 1614],
    "block32":[2450, 1130, 2953, 1473],
    "block33":[3025, 1184, 3828, 1703],
    "block34":[954, 1767, 1201, 2153],
    "block35":[1242, 1743, 1459, 2157],
    "block36":[1522, 1730, 1740, 2154],
    "block37":[1834, 1887, 1900, 1996],
    "block38":[1838, 1995, 1905, 2022],#[1831, 1996, 1905, 2065],
    "block39":[1831, 2058, 1890, 2079],#[1825, 2065, 1904, 2128],
    "block40":[1795, 2134, 1910, 2159],#[1826, 2129, 1907, 2159],
    "block41":[2019, 1820, 2129, 2159],
    "block42":[2364, 1831, 2484, 2156],
    "block43":[2483, 1826, 2604, 2156],
    "block44":[2638, 1811, 2759, 2157],
    "block45":[2982, 1850, 3102, 2158],
}#46, 47 Not Yet

# block_pos = {
#     "block1":[233, 10, 534, 580],
#     "block2":[757, 188, 1338, 571],
#     "block3":[1512, 310, 1893, 669],
#     "block4":[2654, 313, 3444, 763],
#     "block5":[432, 927, 530, 1014],
#     "block6":[425, 1029, 526, 1114],
#     "block7":[430, 1134, 532, 1205],
#     "block8":[422, 1219, 536, 1303],
#     "block9":[424, 1312, 534, 1407],
#     "block10":[542, 940, 638, 1028],
#     "block11":[533, 1034, 635, 1118],
#     "block12":[533, 1133, 636, 1215],
#     "block13":[535, 1215, 634, 1303],
#     "block14":[544, 1305, 641, 1395],
#     "block15":[665, 964, 836, 1374],
#     "block16":[837, 964, 996, 1358],
#     "block17":[1159, 1022, 1261, 1086],
#     "block18":[1154, 1097, 1263, 1166],
#     "block19":[1158, 1181, 1259, 1248],
#     "block20":[1155, 1255, 1260, 1324],
#     "block21":[1263, 1025, 1365, 1088],
#     "block22":[1263, 1098, 1360, 1161],
#     "block23":[1262, 1170, 1360, 1237],
#     "block24":[1262, 1253, 1360, 1320],
#     "block25":[1368, 1031, 1461, 1093],
#     "block26":[1370, 1106, 1456, 1172],
#     "block27":[1368, 1184, 1454, 1239],
#     "block28":[1366, 1254, 1460, 1314],
#     "block29":[1439, 1305, 1475, 1342],
#     "block30":[1473, 1006, 1522, 1340],
#     "block31":[1528, 846, 1731, 1283],
#     "block32":[1932, 818, 2288, 1100],
#     "block33":[2436, 945, 2616, 1247],
#     "block34":[222, 1568, 549, 2151],
#     "block35":[683, 1525, 944, 2044],
#     "block36":[1068, 1490, 1293, 1982],
#     "block37":[1445, 1573, 1548, 1658],
#     "block38":[1447, 1661, 1558, 1728],
#     "block39":[1453, 1721, 1553, 1803],
#     "block40":[1451, 1806, 1556, 1876],
#     "block41":[1608, 1490, 1727, 1811],
#     "block42":[1890, 1458, 1997, 1742],
#     "block43":[1986, 1442, 2080, 1718],
#     "block44":[2072, 1428, 2173, 1705],
#     "block45":[2276, 1417, 2359, 1676],
#     "block46":[2225, 503, 2395, 827]
# }

In [16]:
import cv2
import cv2.ximgproc as xip

def canny_(img, kernel_size, low_thr, high_thr):

    # wmf = xip.weightedMedianFilter(img, img, 10, 30)
    # cv2.imshow('wmf', wmf)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    
    # img_ = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur_gray = cv2.GaussianBlur(img, (kernel_size, kernel_size), 100)
    sharpen = cv2.addWeighted(img, 2.5, blur_gray, 5, -5)

    edges = cv2.Canny(img, low_thr, high_thr)
    return edges
    # pic = Image.open(edges)
    # plt.imshow(edges)
    # plt.show()
    # plt.gcf().canvas.draw()

    # cv2.imshow('edge', edges)
    # cv2.waitKey(0)
    # cv2.imwrite('edge.png', edges)
    # cv2.destroyAllWindows()

In [6]:
def make_data(path, target, goal, coordination, new_dir):
  # print(os.listdir(path+target))
  files = sorted([os.path.join(path+target+goal, x) for x in os.listdir(path+target+goal) if x.endswith(".jpg")])
  print(len(files))
  print(os.listdir(path))
  if new_dir[1:] not in os.listdir(path):
    os.mkdir(path+new_dir)
  try:
    shutil.rmtree(path+new_dir+goal)
  except:
    print(f"The {goal[1:]} set hasn't been created!")
  os.mkdir(path+new_dir+goal)

  zeros = 0
  ones = 0

  for idx, name in enumerate(tqdm(files)):
    pic = Image.open(name)
    pic = pic.crop((coordination[0], coordination[1], coordination[2], coordination[3]))
    
    if "0_" in name:
      pic.save(path+new_dir+goal+f"/0_{zeros+1}.jpg")
      zeros += 1
    elif "1_" in name:
      pic.save(path+new_dir+goal+f"/1_{ones+1}.jpg")
      ones += 1
    else:
      pic.save(path+new_dir+goal+f"/{idx+1}.jpg")
    # pic.save(path+new_dir+goal+"/"+name.split("/", 4)[-1])

In [7]:
print(len(block_pos))

45


In [8]:
coord_values = list(block_pos.values())
blocks = list(block_pos.keys())
print(coord_values)
print(blocks)

[[964, 660, 1182, 1092], [1222, 667, 1713, 1014], [1853, 667, 2359, 1022], [2866, 704, 3115, 1101], [1025, 1332, 1090, 1397], [1017, 1403, 1083, 1470], [1015, 1476, 1092, 1539], [1025, 1545, 1084, 1606], [1013, 1605, 1087, 1675], [1097, 1332, 1184, 1390], [1084, 1394, 1189, 1462], [1098, 1469, 1185, 1539], [1104, 1536, 1188, 1599], [1087, 1603, 1180, 1675], [1209, 1301, 1331, 1667], [1331, 1317, 1451, 1656], [1554, 1368, 1638, 1448], [1561, 1451, 1633, 1519], [1561, 1522, 1631, 1582], [1551, 1583, 1632, 1657], [1652, 1376, 1725, 1446], [1639, 1449, 1731, 1520], [1643, 1514, 1721, 1590], [1647, 1591, 1724, 1653], [1746, 1373, 1812, 1441], [1742, 1443, 1817, 1518], [1745, 1515, 1811, 1587], [1740, 1589, 1809, 1656], [1836, 1336, 1872, 1636], [1873, 1333, 1910, 1631], [1971, 1159, 2190, 1614], [2450, 1130, 2953, 1473], [3025, 1184, 3828, 1703], [954, 1767, 1201, 2153], [1242, 1743, 1459, 2157], [1522, 1730, 1740, 2154], [1834, 1887, 1900, 1996], [1838, 1995, 1905, 2022], [1831, 2058, 1890

In [9]:
path = "./imgs_2nd"
target = "/all" 

for i in tqdm(range(len(blocks))):
  if i == 39:
    new_dir = "/" + blocks[i]
    print(new_dir)
    make_data(path, target, "/testing", coord_values[i], new_dir)
    make_data(path, target, "/training", coord_values[i], new_dir)

  0%|          | 0/45 [00:00<?, ?it/s]

/block40
100
['block40', 'all', 'block33', 'block1', 'block2', 'block3', 'block4', 'block5', 'block6', 'block7', 'block8', 'block9', 'block10', 'block11', 'block12', 'block13', 'block14', 'block15', 'block16', 'block17', 'block18', 'block19', 'block20', 'block21', 'block22', 'block23', 'block24', 'block25', 'block26', 'block27', 'block28', 'block29', 'block30', 'block31', 'block32', 'block34', 'block35', 'block36', 'block37', 'block38', 'block39', 'block41', 'block42', 'block43', 'block44', 'block45']


  0%|          | 0/100 [00:00<?, ?it/s]

500
['block40', 'all', 'block33', 'block1', 'block2', 'block3', 'block4', 'block5', 'block6', 'block7', 'block8', 'block9', 'block10', 'block11', 'block12', 'block13', 'block14', 'block15', 'block16', 'block17', 'block18', 'block19', 'block20', 'block21', 'block22', 'block23', 'block24', 'block25', 'block26', 'block27', 'block28', 'block29', 'block30', 'block31', 'block32', 'block34', 'block35', 'block36', 'block37', 'block38', 'block39', 'block41', 'block42', 'block43', 'block44', 'block45']


  0%|          | 0/500 [00:00<?, ?it/s]